In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

# Pandas Group Operations

Selanjutnya kita akan membahas operasi group pada pandas. Bagian library pandas ini tidak memiliki fitur sebanyak bagian lainnya, dan itu bagus.

* groupby
* agg
* filter
* transform

Lihat dokumentasi lengkapnya [di sini](http://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)

Mari kita mulai dengan dataset tips:

In [2]:
tips = sns.load_dataset('tips')
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


### Groupby

Operasi group by dimulai dengan menentukan kelompok data mana yang ingin kita operasikan. Ada banyak cara untuk membuat grup, tetapi fungsi yang digunakan panda untuk membuat grup data adalah `groupby`

In [17]:
tips_gb = tips.groupby(['sex', 'smoker'])
tips_gb

Groupby bekerja dengan memberi pandas beberapa nama kolom. Pandas akan melihat data dan melihat setiap kombinasi unik dari kolom yang ditentukan. Setiap kombinasi unik adalah grup. Jadi dalam hal ini kita akan memiliki empat kelompok: perokok pria, perokok wanita, pria bukan perokok, wanita bukan perokok.

### Agg

Operasi agregat menggabungkan semua data dalam grup ini menjadi satu nilai. Anda menggunakan dictionary untuk menentukan nilai mana yang diinginkan. Sebagai contoh lihat di bawah ini, kami menanyakan nilai mean dan min dari kolom tips untuk setiap grup:

In [4]:
tips_agg = tips_gb.agg({
    'tip': ['mean', 'min'],
    'day': 'first',
    'total_bill': 'size'
})

tips_agg

tip         day total_bill
                   mean   min first       size
sex    smoker                                 
Male   Yes     3.051167  1.00   Sat         60
       No      3.113402  1.25   Sun         97
Female Yes     2.931515  1.00   Sat         33
       No      2.773519  1.00   Sun         54

Jadi perhatikan bahwa kita mendapatkan multi-indeks untuk indeks dan kolom. Kita selalu dapat menghapus multi-indeks dengan `reset_index`

In [5]:
tips_agg.reset_index()

sex smoker       tip         day total_bill
                      mean   min first       size
0    Male    Yes  3.051167  1.00   Sat         60
1    Male     No  3.113402  1.25   Sun         97
2  Female    Yes  2.931515  1.00   Sat         33
3  Female     No  2.773519  1.00   Sun         54

Dan kita bisa menggunakan stacking atau trik kolom kita untuk menghilangkan kolom yang tidak terpakai:

In [6]:
# sebelum
tips_agg.columns

MultiIndex([(       'tip',  'mean'),
            (       'tip',   'min'),
            (       'day', 'first'),
            ('total_bill',  'size')],
           )

In [7]:
tips_agg.stack()

tip  day  total_bill
sex    smoker                                 
Male   Yes    first       NaN  Sat         NaN
              mean   3.051167  NaN         NaN
              min    1.000000  NaN         NaN
              size        NaN  NaN        60.0
       No     first       NaN  Sun         NaN
              mean   3.113402  NaN         NaN
              min    1.250000  NaN         NaN
              size        NaN  NaN        97.0
Female Yes    first       NaN  Sat         NaN
              mean   2.931515  NaN         NaN
              min    1.000000  NaN         NaN
              size        NaN  NaN        33.0
       No     first       NaN  Sun         NaN
              mean   2.773519  NaN         NaN
              min    1.000000  NaN         NaN
              size        NaN  NaN        54.0

In [8]:
tips_agg.columns = ['__'.join(col).strip() for col in tips_agg.columns.values]
tips_agg.columns

Index(['tip__mean', 'tip__min', 'day__first', 'total_bill__size'], dtype='object')

In [9]:
tips_agg

tip__mean  tip__min day__first  total_bill__size
sex    smoker                                                  
Male   Yes      3.051167      1.00        Sat                60
       No       3.113402      1.25        Sun                97
Female Yes      2.931515      1.00        Sat                33
       No       2.773519      1.00        Sun                54

Itu saja untuk agregasi, anda dapat menemukan beberapa fungsi agregasi umum yang tercantum [di sini](http://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#aggregation)

### Filter
Operasi grup berikutnya adalah filter. Yang ini cukup sederhana, kita memfilter anggota grup yang tidak memenuhi kriteria.

Sebagai contoh, mari kita lihat waktu paling tidak sibuk tempat itu buka. Salah satu cara yang dapat kita lakukan adalah mengecualikan semua waktu di atas median dari analisis

In [10]:
# kita menggunakan sintaks groupby yang sama persis
tips_gb = tips.groupby(['day', 'time'])

In [11]:
median_size = tips_gb.agg({'size': 'sum'}).median()[0]

In [12]:
# perhatikan bahwa kita menghilangkan beberapa baris
tips_gb.filter(lambda group: group['size'].sum() < median_size).head()

,total_bill,tip,sex,smoker,day,time,size
220,12.16,2.20,Male,Yes,Fri,Lunch,2
221,13.42,3.48,Female,Yes,Fri,Lunch,2
222,8.58,1.92,Male,Yes,Fri,Lunch,1
223,15.98,3.00,Female,No,Fri,Lunch,3
224,13.42,1.58,Male,Yes,Fri,Lunch,2


### Transform

Operasi grup terakhir adalah transformasi. Ini menggunakan informasi grup untuk menerapkan transformasi ke titik data individual. Sebagai contoh lihat di bawah ini: setiap hari mari kita bagi dengan tagihan dan tip dengan jumlah rata-rata yang dihabiskan pada hari itu. Dengan begitu kita dapat melihat seberapa besar perbedaan tagihan itu dari rata-rata hari itu

In [13]:
tips_gb = tips.groupby(['day'])

In [14]:
tips_gb[['total_bill', 'tip']].transform(lambda x: x / x.mean()).head()

,total_bill,tip
0,0.793554,0.310279
1,0.482952,0.509964
2,0.981317,1.075225
3,1.106025,1.016856
4,1.148529,1.109018
